In [1]:
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import plotly.graph_objects as go
import matplotlib.pylab as plt
from astropy.time import Time
from time import time


from multiprocessing import Process
import os

# print(f"{np.__name__} --> version {np.__version__}")
# print(f"{pa.__name__} --> version {pa.__version__}")
!python -c "import numpy as xx; print(f'{xx.__name__} \t--> version {xx.__version__}')"
!python -c "import pyarrow as xx; print(f'{xx.__name__} --> version {xx.__version__}')"
!python -c "import plotly as xx; print(f'{xx.__name__} \t--> version {xx.__version__}')"
!python -c "import astropy as xx; print(f'{xx.__name__} --> version {xx.__version__}')"


numpy 	--> version 1.22.4
pyarrow --> version 9.0.0
plotly 	--> version 5.10.0
astropy --> version 5.1


In [9]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Universidad/Trabajo de Titulo/Modelos ML curvas de luz

# %cd /content/drive/MyDrive/Universidad/Trabajo de Titulo/Modelos ML curvas de luz#Aca va tu ruta del archivo en tu drive

ModuleNotFoundError: No module named 'google.colab'

In [2]:
def getStatistic(index,indBin,magnitud, amount_bins):
    #Analisis en base a la magnitud
    mag_ordered = magnitud[index]
    # print('mag index',index)
    # print('mag',mag_ordered)
    #Obtención de vectores con caracteristicas estadisticas
    
    vector_mean = np.array([np.nanmean(mag_ordered[indBin==(i+1)])  if (indBin==(i+1)).sum()>0 else -1000  for i in range(amount_bins)])
    vector_max = np.array([np.nanmax(mag_ordered[indBin==(i+1)])  if (indBin==(i+1)).sum()>0 else -1000 for i in range(amount_bins)])
    vector_min = np.array([np.nanmin(mag_ordered[indBin==(i+1)])  if (indBin==(i+1)).sum()>0 else -1000 for i in range(amount_bins)])
    vector_std = np.array([np.nanstd(mag_ordered[indBin==(i+1)])  if (indBin==(i+1)).sum()>0 else -1000 for i in range(amount_bins)])

    # mean_process = Process(target=np.array([np.nanmean(mag_ordered[indBin==(i+1)])  if (indBin==(i+1)).sum()>0 else -1000  for i in range(amount_bins)]))
    # max_process = Process(target=np.array([np.nanmax(mag_ordered[indBin==(i+1)])  if (indBin==(i+1)).sum()>0 else -1000 for i in range(amount_bins)]))
    # min_process = Process(target=np.array([np.nanmin(mag_ordered[indBin==(i+1)])  if (indBin==(i+1)).sum()>0 else -1000 for i in range(amount_bins)]))
    # std_process = Process(target=np.array([np.nanstd(mag_ordered[indBin==(i+1)])  if (indBin==(i+1)).sum()>0 else -1000 for i in range(amount_bins)]))

    #start multiprocessing 
    # mean_process.start()
    # max_process.start()
    # min_process.start()
    # std_process.start()

    # get data
    # vector_mean = mean_process.join()
    # vector_max = max_process.join()
    # vector_min = min_process.join()
    # vector_std = std_process.join()
    # return [vector_mean,vector_max,vector_min,vector_std]
    return [vector_mean,vector_max,vector_min,vector_std]

    

def drawGraphic(bins,time,mag,indice,period):
    title_figure = f'Folded Time Serie with period T/{period}'
    #Grafico para ver la serie de tiempo 
    plt.figure(figsize=(12,4))
    plt.plot(time[indice], mag[indice])
    plt.scatter(time[indice], mag[indice])
    plt.ylabel("Magnitud")
    plt.xlabel("Time")
    plt.suptitle(title_figure, fontsize=22)
    plt.grid(axis='x', color='red', linestyle='-', linewidth=2)

    for bin in bins:
        plt.axvline(x=bin,color='red')
    # plt.grid(axis='y', color='gray', linestyle='dashed')
    plt.show()


def variantion_period_v2(array_periods,ts_time, ts_mag, amount_bins=10):
    tensor_data = []
    for period in array_periods:
        #valor del periodo en decimales
        period_float = 1 / period
        #proceso de folding
        folded_time = ts_time % period_float
        # print('folded_time',folded_time)

        #indices de los datos para encontrarlos luego del folding
        index_folded = np.argsort(folded_time)
        # print('index_folded',index_folded)

        #Divido el periodo en bins o intervalos [0,n] para luego obtener datos estadisticos de cada bin
        bins = np.linspace(0,period_float,amount_bins+1)
        #Concentración de datos en cada bin
        concentration, bin_edges= np.histogram(folded_time[index_folded],bins=bins)
        #indices - Devuelve los índices de los bins a los que pertenece cada valor del folded_time
        index_bin = np.digitize(folded_time[index_folded],bins=bins)

        # print("bins: ",bins)
        # print("concentracion de datos en cada bin: ",concentration)
        #Obtención de datos estadisticos de cada bin y formación del tensor
        vectorStatisticPeriod = np.asmatrix(getStatistic(index_folded,index_bin,ts_mag,amount_bins))
        tensor_data.append(vectorStatisticPeriod)

    return tensor_data

# funcion para graficar la serie de tiempo con link al sitio web de ALeRCE del evento
def GraphicLightCurve(fig,band,light_curve,show):
    mtcolor = [['g','green'],['r','red'],['b','blue']]
    idx=(light_curve['fid']==band).values
    fig.add_trace(
            go.Scatter(
                x = light_curve.mjd.values[idx],
                y = light_curve.magpsf_corr.values[idx],
                mode = 'markers', name=mtcolor[band-1][0],
                marker_color = mtcolor[band-1][1],
                error_y = dict(type='data', array=light_curve.sigmapsf_corr.values[idx], visible=True),
                marker = dict(size=10, opacity=0.5, showscale=False, line=dict(width=1)), 
                showlegend = True,
                legendgroup = mtcolor[band-1][0],
                hovertemplate ='<b>' + mtcolor[band-1][0] + ':\t\t</b>' + light_curve.magpsf_corr[idx].map('{:.3f}'.format).apply(str) + ' ± ' +
                                light_curve.sigmapsf_corr[idx].map('{:.3f}'.format).apply(str)  + '<br>' + 
                                '<b>MJD:\t</b>'+ light_curve.mjd[idx].apply(str)  + '<br>' + 
                                '<b>Date:\t</b>'+ [temp.strftime('%a, %d %b %Y %H:%M:%S UTC') for temp in Time(light_curve.mjd[idx].values, format='mjd').datetime]+ '<br>' +
                                '<extra></extra>'
                ))
    # fig['layout']['yaxis']['autorange'] = "reversed"
    fig.update_xaxes(
        tickangle=0,
        title_text="Modified Julian Dates [MJD]",
        title_font={"size": 14})
    
    fig.update_yaxes(
        autorange= 'reversed',
        tickangle=0,
        title_text="Magnitude [Mag]",
        title_font={"size": 14})
    
    fig.update_layout(
        title={'text':'<b>Light Curve</b>' + '<br>' +
                        '<a href="https://alerce.online/object/'+ event + '">Object: ' + event + 'text</a>',
                'font':dict(size=14),'x':0.5, 'y':0.9,'xanchor': 'center','yanchor': 'top'})
    
    if_show_fig = lambda show: fig.show() if show == True else False
    if_show_fig(show)


In [3]:
parquet_file = pq.ParquetFile('../lightcurves_ts.parquet')
# parquet_file.schema_arrow.names # obtienes nombre columnas
# parquet_file.metadata.num_rows # numero de filas
# parquet_file.metadata.num_columns # nummero de columnas 

print("numero de columnas:", parquet_file.metadata.num_columns)
print("numero de filas (datos): ", parquet_file.metadata.num_rows)
# print("nombre columnas: ", parquet_file.schema_arrow.names)


numero de columnas: 113
numero de filas (datos):  13844917


In [5]:
# Descipcion de las columnas
# oid -- object id
# fid -- filter id, filtro o banda utilizada (verde,rojo,azul,etc)
# mjd -- medida de tiempo continua (Modified Julian date / Calendario Juliano Modificado)
# magpsf_corr -- magnitud psf correjida / posicion en el espacio, es el promedio de los pixeles de una deteccion
# sigmapsf_corr -- variacion de los pixeles de una deteccion
# classALeRCE - clasificación que realiza ALeRCE en base a los parametros obtenidos (152 aprox) con el random forest de 2 niveles

# se divide el paquete en grupos --> se ocupa el primer grupo 0
data = parquet_file.read_row_group(0, ['oid', 'fid', 'mjd', 'magpsf_corr', 'sigmapsf_corr', 'classALeRCE']).to_pandas()

In [6]:
data.head()

,oid,fid,mjd,magpsf_corr,sigmapsf_corr,classALeRCE
0,ZTF17aaacfxu,1,58863.138021,20.996371,0.444790,RRL
1,ZTF17aaacfxu,1,58867.179803,19.374605,0.175861,RRL
2,ZTF17aaacfxu,1,58871.170336,19.630466,0.123111,RRL
3,ZTF17aaacfxu,1,58880.146204,19.362001,0.103466,RRL
4,ZTF17aaacfxu,1,58849.131389,19.530453,0.078387,RRL


In [7]:
# muestra los eventos unicos, sin repeticiones
all_event = data.oid.unique()

print('eventos totales',all_event.shape)

eventos totales (173693,)


Probando cosas


In [8]:
# data.head()

# limpieza de datos 
df_data = data.replace([np.inf, -np.inf], np.nan)
df_data = df_data.dropna()
df_data.head()


print(df_data['classALeRCE'].values.size)
df_data.head()

13697564


,oid,fid,mjd,magpsf_corr,sigmapsf_corr,classALeRCE
0,ZTF17aaacfxu,1,58863.138021,20.996371,0.444790,RRL
1,ZTF17aaacfxu,1,58867.179803,19.374605,0.175861,RRL
2,ZTF17aaacfxu,1,58871.170336,19.630466,0.123111,RRL
3,ZTF17aaacfxu,1,58880.146204,19.362001,0.103466,RRL
4,ZTF17aaacfxu,1,58849.131389,19.530453,0.078387,RRL


In [26]:
df_data_unique = df_data.oid.unique()
limit = 10
i = 0
all_time_serie = []

time_test = time()

for index, name_event in enumerate(df_data_unique):
    event = df_data.loc[df_data['oid'] == name_event].sort_values(by=['mjd'])
    rows_event, cols_event = event.shape

    if(rows_event == 1):
        continue

    all_time_serie.append(event)
    print(event)
    print('-'*100)

    i = i + 1

    if i == limit:
        break
time_end = time() - time_test

print(f'time: {time_end}')
print(len(all_time_serie))




            oid  fid           mjd  magpsf_corr  sigmapsf_corr classALeRCE
4  ZTF17aaacfxu    1  58849.131389    19.530453       0.078387         RRL
0  ZTF17aaacfxu    1  58863.138021    20.996371       0.444790         RRL
1  ZTF17aaacfxu    1  58867.179803    19.374605       0.175861         RRL
2  ZTF17aaacfxu    1  58871.170336    19.630466       0.123111         RRL
3  ZTF17aaacfxu    1  58880.146204    19.362001       0.103466         RRL
----------------------------------------------------------------------------------------------------
             oid  fid           mjd  magpsf_corr  sigmapsf_corr classALeRCE
10  ZTF17aaaemke    1  58372.401910    16.530643       0.014738       EB/EW
45  ZTF17aaaemke    1  58388.324433    16.681208     100.000000       EB/EW
29  ZTF17aaaemke    1  58390.353206    16.589474       0.008474       EB/EW
58  ZTF17aaaemke    2  58431.186701    15.361944     100.000000       EB/EW
46  ZTF17aaaemke    1  58431.211875    16.109080     100.000000      

In [29]:
# ----------------------------
limit_event = 1000
num_periods = 4
nbines = 10
array_periods = 2**(np.arange(num_periods)+0)

tensor_all_event = {}
time_arr = []
# ----------------------------

main_start = time()

for index, event in enumerate(all_time_serie):
    event_start = time()
    # obtiene el nombre del evento
    name_unique = event.oid.unique()
    tensor_band = {}

    for band_fid in range(1,4):
        event_band = event.loc[event['fid'] == band_fid].sort_values(by=['mjd'])
        
        if(event_band.empty is False):
            rows, cols = event_band.shape

            if(rows == 1):
                break

            # normalized
            min_mjd = event_band['mjd'].values.min()
            max_mjd = event_band['mjd'].values.max()
            time_normalized = (event_band['mjd'].values - min_mjd)/(max_mjd - min_mjd)

            # tensor con caracteristicas de cada filtro
            tensor = np.array(variantion_period_v2(array_periods, time_normalized, event_band['magpsf_corr'].values, nbines))
            # tensor de cada banda
            tensor_band[f'Filter {band_fid}'] = tensor
        else:
            break
    
    # asigna el tensor de cada filtro al evento
    tensor_all_event[f'{name_unique[0]}'] = tensor_band
    event_time = time() - event_start
    time_arr.append(event_time)

    if index == limit_event:
        break

print('-'*100)
main_time = time() - main_start
print(f'Time total: {main_time}')
print(f'Total event: {limit_event}')







print(f'Mean Time for each event: {np.mean(time_arr)}')
# print(dict_all_tensor)



----------------------------------------------------------------------------------------------------
Time total: 0.0912177562713623
Total event: 1000
Mean Time for each event: 0.009021782875061035


In [30]:
import json
from json import JSONEncoder

class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)

output = json.dumps(tensor_all_event, cls=NumpyArrayEncoder)

with open('group0.json', 'w') as outfile:
    outfile.write(output)





fin test